<a href="https://colab.research.google.com/github/spring-board-b2-hate-speech/Group-8/blob/madhuri/Comparing_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
#!pip install datasets pandas transformers tensorflow scikit-learn matplotlib

import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
#from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

# Function to encode data using BERT tokenizer
def encode_data(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='tf',
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

# Function to build BERT-based classification model
def build_bert_model(max_len, bert_model_name='bert-base-uncased'):
    # Load pre-trained BERT model and tokenizer
    bert_model = TFBertModel.from_pretrained(bert_model_name)
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)

    # Define inputs
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # BERT encoder
    bert_output = bert_model(input_ids, attention_mask=attention_mask)
    sequence_output = bert_output.last_hidden_state

    # Adding custom layers on top of BERT
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(sequence_output)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Define the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    return model, tokenizer

# Loading and preprocessing

df = pd.read_csv("/content/Preprocessed_cleaned_Final_dataset.csv")

# Reduce dataset size for faster training
df = df.sample(n=10)

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2)

max_len = 128
batch_size = 32
epochs = 1

train_texts = train_df['Text'].tolist()  #Contains text data
train_labels = train_df['IsHatespeech'].tolist()   # Contains labels
test_texts = test_df['Text'].tolist()
test_labels = test_df['IsHatespeech'].tolist()

#string labels to numerical labels (assuming binary classification)
train_labels = [0 if label == '0' else 1 for label in train_labels]
test_labels = [0 if label == '0' else 1 for label in test_labels]

model, tokenizer = build_bert_model(max_len)

train_input_ids, train_attention_masks = encode_data(train_texts, tokenizer, max_len)
test_input_ids, test_attention_masks = encode_data(test_texts, tokenizer, max_len)

train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

# Compiling
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model Training
history = model.fit(
    [train_input_ids, train_attention_masks], train_labels,
    validation_data=([test_input_ids, test_attention_masks], test_labels),
    epochs=epochs,
    batch_size=batch_size
)

# Model eval
results = model.evaluate([test_input_ids, test_attention_masks], test_labels)

print(results)

# Predictions for classification report
test_predictions = model.predict([test_input_ids, test_attention_masks])
test_predictions = np.round(test_predictions).astype(int).flatten()

# Classification report
print("Deep Learning Model Performance:")
print("Classification Report:\n", classification_report(test_labels, test_predictions))
print("Accuracy:", accuracy_score(test_labels, test_predictions))
print("AUC-ROC:", roc_auc_score(test_labels, test_predictions))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_predictions))
print("F1 Score:", f1_score(test_labels, test_predictions))
print("Precision:", precision_score(test_labels, test_predictions))
print("Recall:", recall_score(test_labels, test_predictions))

#training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')

plt.show()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

1/1 [==============================] - 1s 821ms/step - loss: 0.6564 - accuracy: 1.0000
[0.6563659906387329, 1.0]
1/1 [==============================] - 13s 13s/step
Deep Learning Model Performance:
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

Accuracy: 1.0


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Combine text data for tokenization and word2vec training
texts = pd.concat([train_df['clean_text'], test_df['clean_text']]).tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(train_df['clean_text'])
sequences_test = tokenizer.texts_to_sequences(test_df['clean_text'])

# Get vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences
max_seq_length = max(max([len(seq) for seq in sequences_train]), max([len(seq) for seq in sequences_test]))
X_train_pad = pad_sequences(sequences_train, maxlen=max_seq_length)
X_test_pad = pad_sequences(sequences_test, maxlen=max_seq_length)

# Train Word2Vec model
sentences = [text.split() for text in texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Prepare labels
y_train = train_df['IsHatespeech'].values
y_test = test_df['IsHatespeech'].values

# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Train the model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=128, validation_data=(X_test_pad, y_test), verbose=2, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict and evaluate performance
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score

print("Deep Learning Model Performance with Word2Vec:")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Epoch 1/5
59/59 - 95s - loss: 0.6255 - accuracy: 0.6469 - val_loss: 0.6206 - val_accuracy: 0.6622 - lr: 0.0010 - 95s/epoch - 2s/step
Epoch 2/5
59/59 - 102s - loss: 0.5949 - accuracy: 0.6760 - val_loss: 0.6264 - val_accuracy: 0.6369 - lr: 0.0010 - 102s/epoch - 2s/step
Epoch 3/5
59/59 - 115s - loss: 0.6113 - accuracy: 0.6466 - val_loss: 0.5895 - val_accuracy: 0.6799 - lr: 0.0010 - 115s/epoch - 2s/step
Epoch 4/5
59/59 - 82s - loss: 0.5803 - accuracy: 0.6869 - val_loss: 0.5827 - val_accuracy: 0.6880 - lr: 0.0010 - 82s/epoch - 1s/step
Epoch 5/5
59/59 - 81s - loss: 0.5795 - accuracy: 0.6923 - val_loss: 0.5990 - val_accuracy: 0.6729 - lr: 0.0010 - 81s/epoch - 1s/step
Test Accuracy: 0.6729323267936707
59/59 [==============================] - 4s 64ms/step
Deep Learning Model Performance with Word2Vec:
Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.66      0.67       944
           1       0.66      0.69      0.67       918

    accu

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Combine text data for tokenization and word2vec training
texts = pd.concat([train_df['clean_text'], test_df['clean_text']]).tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(train_df['clean_text'])
sequences_test = tokenizer.texts_to_sequences(test_df['clean_text'])

# Get vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences
max_seq_length = max(max([len(seq) for seq in sequences_train]), max([len(seq) for seq in sequences_test]))
X_train_pad = pad_sequences(sequences_train, maxlen=max_seq_length)
X_test_pad = pad_sequences(sequences_test, maxlen=max_seq_length)

# Train Word2Vec model
sentences = [text.split() for text in texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Prepare labels
y_train = train_df['IsHatespeech'].values
y_test = test_df['IsHatespeech'].values

# Build the model with more neurons and additional Dense layers
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))  # Increased neurons and dropout
model.add(Dense(128, activation='relu'))  # Increased neurons
model.add(Dropout(0.3))  # Increased dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Train the model
history = model.fit(X_train_pad, y_train, epochs=1, batch_size=128, validation_data=(X_test_pad, y_test), verbose=2, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

# Predict and evaluate performance
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score

print("Deep Learning Model Performance with Word2Vec:")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))


59/59 - 163s - loss: 0.6284 - accuracy: 0.6351 - val_loss: 0.6064 - val_accuracy: 0.6762 - lr: 0.0010 - 163s/epoch - 3s/step
Test Accuracy: 0.6761546730995178
59/59 [==============================] - 17s 284ms/step
Deep Learning Model Performance with Word2Vec:
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.53      0.63       944
           1       0.63      0.82      0.71       918

    accuracy                           0.68      1862
   macro avg       0.69      0.68      0.67      1862
weighted avg       0.69      0.68      0.67      1862

Accuracy: 0.6761546723952739
AUC-ROC: 0.7365011447140061
Confusion Matrix:
 [[504 440]
 [163 755]]
F1 Score: 0.7146237576904875
Precision: 0.6317991631799164
Recall: 0.8224400871459695


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Combine text data for tokenization and word2vec training
texts = pd.concat([train_df['clean_text'], test_df['clean_text']]).tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(train_df['clean_text'])
sequences_test = tokenizer.texts_to_sequences(test_df['clean_text'])

# Get vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences
max_seq_length = max(max([len(seq) for seq in sequences_train]), max([len(seq) for seq in sequences_test]))
X_train_pad = pad_sequences(sequences_train, maxlen=max_seq_length)
X_test_pad = pad_sequences(sequences_test, maxlen=max_seq_length)

# Train Word2Vec model
sentences = [text.split() for text in texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Prepare labels
y_train = train_df['IsHatespeech'].values
y_test = test_df['IsHatespeech'].values

# Split the data
X = train_df['clean_text']
y = train_df['IsHatespeech']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Tokenize and pad sequences for the training and validation data
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_val = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(sequences_train, maxlen=max_seq_length)
X_val_pad = pad_sequences(sequences_val, maxlen=max_seq_length)

# Resample (oversample) the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_pad, y_train)

# Build the model with more neurons and additional Dense layers
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))  # Increased neurons and dropout
model.add(Dense(128, activation='relu'))  # Increased neurons
model.add(Dropout(0.3))  # Increased dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Train the model
history = model.fit(X_train_res, y_train_res, epochs=5, batch_size=128, validation_data=(X_val_pad, y_val), verbose=2, callbacks=[early_stopping, reduce_lr])


Epoch 1/5
48/48 - 217s - loss: 0.6570 - accuracy: 0.5820 - val_loss: 0.6065 - val_accuracy: 0.6582 - lr: 0.0010 - 217s/epoch - 5s/step
Epoch 2/5
48/48 - 137s - loss: 0.6214 - accuracy: 0.6614 - val_loss: 0.6285 - val_accuracy: 0.6286 - lr: 0.0010 - 137s/epoch - 3s/step
Epoch 3/5
48/48 - 130s - loss: 0.6154 - accuracy: 0.6624 - val_loss: 0.6065 - val_accuracy: 0.6521 - lr: 0.0010 - 130s/epoch - 3s/step
Epoch 4/5
48/48 - 130s - loss: 0.5939 - accuracy: 0.6800 - val_loss: 0.5629 - val_accuracy: 0.7045 - lr: 2.0000e-04 - 130s/epoch - 3s/step
Epoch 5/5
48/48 - 146s - loss: 0.5924 - accuracy: 0.6875 - val_loss: 0.5638 - val_accuracy: 0.7119 - lr: 2.0000e-04 - 146s/epoch - 3s/step


In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")


y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score

print("Deep Learning Model Performance with Word2Vec:")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Test Accuracy: 0.6769067645072937
59/59 [==============================] - 19s 309ms/step
Deep Learning Model Performance with Word2Vec:
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.55      0.63       944
           1       0.64      0.81      0.71       918

    accuracy                           0.68      1862
   macro avg       0.69      0.68      0.67      1862
weighted avg       0.70      0.68      0.67      1862

Accuracy: 0.6793770139634802
AUC-ROC: 0.7450380340460102
Confusion Matrix:
 [[518 426]
 [171 747]]
F1 Score: 0.7144906743185079
Precision: 0.6368286445012787
Recall: 0.8137254901960784


In [ ]:
#!pip install tensorflow
import tensorflow as tf # Import tensorflow
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tqdm.keras import TqdmCallback

# Load dataset
train_df = pd.read_csv('Train_data.csv')
test_df = pd.read_csv('Test_data.csv')

# Combine
texts = pd.concat([train_df['clean_text'], test_df['clean_text']]).tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(train_df['clean_text'])
sequences_test = tokenizer.texts_to_sequences(test_df['clean_text'])

vocab_size = len(tokenizer.word_index) + 1

max_seq_length = max(max([len(seq) for seq in sequences_train]), max([len(seq) for seq in sequences_test]))
X_train_pad = pad_sequences(sequences_train, maxlen=max_seq_length)
X_test_pad = pad_sequences(sequences_test, maxlen=max_seq_length)

sentences = [text.split() for text in texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

y_train = train_df['IsHatespeech'].values
y_test = test_df['IsHatespeech'].values

y_train = y_train.astype(np.float32)  # Cast to float32
y_test = y_test.astype(np.float32)  # Cast to float32

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
model.add(Bidirectional(LSTM(1024, return_sequences=True)))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Bidirectional(LSTM(256, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=[tf.keras.metrics.F1Score(dtype='float32')])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

# Fit the model with tqdm progress bar
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=128, validation_data=(X_test_pad, y_test), verbose=0, callbacks=[early_stopping, reduce_lr, TqdmCallback(verbose=1)])

loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score

print("Deep Learning Model Performance with Word2Vec:")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_prob))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Test Accuracy: [0.6604317]
59/59 [==============================] - 14s 219ms/step
Deep Learning Model Performance with Word2Vec:
Classification Report:
               precision    recall  f1-score   support

         0.0       0.66      0.67      0.66       944
         1.0       0.66      0.65      0.65       918

    accuracy                           0.66      1862
   macro avg       0.66      0.66      0.66      1862
weighted avg       0.66      0.66      0.66      1862

Accuracy: 0.658968850698174
AUC-ROC: 0.7164346082123999
Confusion Matrix:
 [[630 314]
 [321 597]]
F1 Score: 0.6528157463094588
Precision: 0.6553238199780461
Recall: 0.6503267973856209
